<a href="https://colab.research.google.com/github/thanhtrinh2003/GarbageClassificationKaggle/blob/main/Method_1_Pre_trained_model_predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import important libraries


In [ ]:
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import AveragePooling2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.convolutional import ZeroPadding2D
from keras.layers.core import Activation
from keras.layers.core import Dense
from keras.layers import Flatten
from keras.layers import Input
from keras.models import Model
from keras.layers import add
from keras.regularizers import l2
from keras import backend as K
from keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD

from __future__ import print_function
import tensorflow.keras
from tensorflow.keras.layers import Dense, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation
from tensorflow.keras.layers import AveragePooling2D, Input, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import cifar10
import numpy as np
import os

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

from sklearn import model_selection
import tensorflow as tf
import cv2 as cv
import numpy as np
from google.colab.patches import cv2_imshow

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np



#Import and preprocess the datasets


The dataset is from Kaggle and can be found in the link: https://www.kaggle.com/asdasdasasdas/garbage-classification 

In this notebook, I have already downdloaded the datasets and uploaded on my google drive. First, lets run the code below to mount the drive order to gain access to the datasets stored in Google Drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


After finishing mounting the drives, we will turn our sets of images into a list of arrays [resized_array, class_num] stored in `data`

## Class index
1. cardboard: 0 (contains 403 images)
2. glass: 1 (contains 501 images)
3. plastic: 2 (contains 482 images)
4. paper: 3 (contains 594 images)
5. metal: 4 (contains 410 images)
6. trash: 5 (contains 137 images)

All of our images will be included in `data` with its class number indicated above. 

In [ ]:
labels = ['cardboard', 'glass','plastic','paper','metal','trash']
img_size = 224
data_dir ='/content/drive/MyDrive/Lumiere 2021/Code/Garbage classification/'
data = []

nb = {'cardboard':404, 'glass': 502, 'plastic': 483 , 'paper': 595 , 'metal': 411, 'trash':138}

for label in labels:
  path = data_dir + label + "/" +label
  class_num = labels.index(label)
  for i in tqdm(range(1,nb[label])):
    img_arr = cv.imread(path+str(i)+".jpg")[...,::-1]
    resized_arr = cv.resize(img_arr, (img_size, img_size))
    data.append([resized_arr, class_num])

100%|██████████| 137/137 [00:40<00:00,  3.40it/s]


## Divide `data` into  training, validation, and test datas 

- 15% of the total datasets will be our testing datas
- 85% of the total datasets will be our training and validations ones
  -  15% of these images will be our validations datas
  -  The rest will be training datas

In [ ]:
train, test = model_selection.train_test_split(data, test_size = 0.15)
train, val = model_selection.train_test_split(train, test_size = 0.15/0.85)

Next, we will seperated the data into x and y parts for the convenient for training datas

In [ ]:
x = []
y = []

# x_train = []
# y_train = []
# x_val = []
# y_val = []
# x_test = []
# y_test = []

for feature, label in data:
  x.append(feature)
  y.append(label)

# for feature, label in train:
#   x_train.append(feature)
#   y_train.append(label)

# for feature, label in val:
#   x_val.append(feature)
#   y_val.append(label)

# for feature, label in test:
#   x_test.append(feature)
#   y_test.append(label)

# Normalize the data
# x = np.array(x) / 255
# x_train = np.array(x_train) / 255
# x_val = np.array(x_val) / 255
# x_test = np.array(x_test) / 255

x = np.array(x) 
x.reshape(-1, img_size, img_size, 1)
y = np.array(y)

# x_train.reshape(-1, img_size, img_size, 1)
# y_train = np.array(y_train)

# x_val.reshape(-1, img_size, img_size, 1)
# y_val = np.array(y_val)

# x_test.reshape(-1, img_size, img_size, 1)
# y_test = np.array(y_test)

#Predict with Pretrained ResNet50 models from Keras

First, we will try to predict the garbage class based on the pretrained ResNet50 using ImageNet labels

We will try to connect the labels of ImageNet to match our 6 labels. Initially, we will expect the following:
1. cardboard = carton 
2. glass = wine_bottle, bear_glasses
3. plastic = water_bottle
4. paper = binder, book_jacket, comic_book, envelope, notebook, menu, paper_towel
5. metal = trash_can 
6. trash = ?

In order to have better matching, we will apply ResNet50 model to our datasets and record all the predicted labels. We will look over the labels and added more ImageNet labels to the Kaggle one. 





## Import ResNet Model

From Keras libraries, we will initallize a model with the trained weights from ImageNet

In [ ]:
model = ResNet50(weights = 'imagenet')

102973440/102967424 [==============================] - 1s 0us/step


## Analyzing the ImageNet label

After careful choosing, we decide the correct ImageNet label for the Kaggle:
- cardboard: carton
- glass: beer_bottle, beer_glass, (binoculars), pop_bottle, red_wine, (spot_light), syringe, water_jug, wine_bottle, Petri_dish, (hourglass), perfume,  
- plastic: water_bottle, plastic_bag, Petri_dish, bottle_cap(maybe metal), nipple, (syringe),  
- paper: paper_towel, menu , envelope, comic_book, book_jacket, packet, paper_towel, toilet_tissue, 
- metal: bottlecap, combination_lock, Crock_Pot, (accordion), (bolo_tie), can_opener, corkscrew, (drum), (fountain_pen), hook, shield
- trash: trash will be the left.


In [ ]:
cardboard_list = {'carton'}

glass_list = {'beer_bottle', 'beer_glass', 'pop_bottle', 'red_wine', 'syringe', 'water_jug', 'wine_bottle', 'Petri_dish', 'hourglass', 'perfume'}

plastic_list = {'water_bottle', 'plastic_bag', 'Petri_dish', 'bottle_cap', 'nipple', 'syringe'}

paper_list = {"paper_towel", "menu", "envelope", "comic_book", "book_jacket", "packet", "paper_towel", "toilet_tissue"}

metal_list = {'bottlecap', 'combination_lock', 'Crock_Pot', 'can_opener', 'cockscrew', 'hook', 'shield'}






Now we have chosen the correct ImageNet label for our Kaggle labels. We will try to access the accuracy of Resnet50 based on Kaggle classification model

## Accuracy using Pretrained Model



In [ ]:
current = np.expand_dims(x[i], axis=0)
current = preprocess_input(current) 
preds = model.predict(current)
top1 = decode_predictions(preds, top=5)
print(top1[0][1][1])

studio_couch


In [ ]:
cardboard_correct = 0
glass_correct = 0
paper_correct = 0
plastic_correct = 0
metal_correct = 0
trash_correct = 0
cardboard_correct_5 = 0
glass_correct_5 = 0
paper_correct_5 = 0
plastic_correct_5 = 0
metal_correct_5 = 0
trash_correct_5 = 0



for i in range(x.shape[0]):
  current = np.expand_dims(x[i], axis=0)
  current = preprocess_input(current) 
  preds = model.predict(current)
  top5 = decode_predictions(preds, top=5)
  label = top5[0][0][1]
  #cardboard 
  if y[i] == 0:
    if label in cardboard_list:
      cardboard_correct = cardboard_correct +1
    for i in range(5):
      if top5[0][i][1] in cardboard_list:
        cardboard_correct_5 = cardboard_correct_5 +1
        break;

  #glass
  if y[i] == 1:
    if label in glass_list:
      glass_correct = glass_correct +1
    for i in range(5):
      if top5[0][i][1] in glass_list:
        glass_correct_5 = glass_correct_5 +1
        break;

  #paper
  if y[i] == 2:
    if label in paper_list:
      paper_correct = paper_correct +1
    for i in range(5):
      if top5[0][i][1] in paper_list:
        paper_correct_5 = paper_correct_5 +1
        break;

  #plastic
  if y[i] == 3:
    if label in plastic_list:
      plastic_correct = plastic_correct +1
    for i in range(5):
      if top5[0][i][1] in plastic_list:
        plastic_correct_5 = plastic_correct_5 +1
        break;

  #metal
  if y[i] == 4:
    if label in metal_list:
      metal_correct = metal_correct +1
    for i in range(5):
      if top5[0][i][1] in metal_list:
        metal_correct_5 = metal_correct_5 +1
        break;
  
  #trash
  if y[i] == 5:
    if not (label in metal_list) and not (label in cardboard_list) and not (label in paper_list) and not (label in plastic_list) and not (label in glass_list):
      trash_correct = trash_correct +1
    for i in range(5):
      current = top5[0][i][1]
      if not (current in metal_list) and not (current in cardboard_list) and not (current in paper_list) and not (current in plastic_list) and not (current in glass_list):
        trash_correct_5 = trash_correct_5 +1
        break;



In [ ]:
top1_acc = (cardboard_correct+ glass_correct + paper_correct + plastic_correct + metal_correct + trash_correct) / 2527
top5_acc = (cardboard_correct_5+ glass_correct_5 + paper_correct_5 + plastic_correct_5 + metal_correct_5 + trash_correct_5) / 2527


In [ ]:
print("Top 1 accuracy: " + str(top1_acc))

Top 1 accuracy: 0.22793826671943015


In [ ]:
print("Top 5 accuracy: " + str(top5_acc))

Top 5 accuracy: 0.7340720221606648


In [ ]:
print("Top 1 cardboard: " + str(cardboard_correct/nb['cardboard']))
print("Top 5 cardboard: "+ str(cardboard_correct_5/nb['cardboard']))

Top 1 cardboard: 0.5371287128712872
Top 5 cardboard: 0.8168316831683168


In [ ]:
print("Top 1 glass: " + str(glass_correct/nb['glass']))
print("Top 5 glass: " + str(glass_correct_5/nb['glass']))

Top 1 glass: 0.3645418326693227
Top 5 glass: 0.6374501992031872


In [ ]:
print("Top 1 plastic: " + str(plastic_correct/nb['plastic']))
print("Top 5 plastic: " + str(plastic_correct_5/nb['plastic']))

Top 1 plastic: 0.022774327122153208
Top 5 plastic: 0.10973084886128365


In [ ]:
print("Top 1 paper: " + str(paper_correct/nb['paper']))
print("Top 5 paper: " + str(paper_correct_5/nb['paper']))

Top 1 paper: 0.040336134453781515
Top 5 paper: 0.15126050420168066


In [ ]:
print("Top 1 metal: " + str(metal_correct/nb['metal']))
print("Top 5 metal: " + str(metal_correct_5/nb['metal']))

Top 1 metal: 0.12652068126520682
Top 5 metal: 0.36253041362530414


In [ ]:
print("Top 1 trash: " + str(trash_correct/nb['trash']))
print("Top 5 trash: " + str(trash_correct_5/nb['trash']))

Top 1 trash: 0.644927536231884
Top 5 trash: 0.9782608695652174
